# Imports

In [1]:
!pip install -q \
    torch \
    torchvision \
    torchaudio \
    transformers \
    datasets \
    nltk \
    faiss-cpu \
    matplotlib \
    tqdm \
    numpy \
    pandas \
    accelerate


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
import os
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Optional
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from nltk.translate.bleu_score import sentence_bleu
import nltk
import faiss
import torch.nn.functional as F
from torch.utils.data import Dataset

import os
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm import tqdm

import itertools
import time
import warnings

# Chargement des données

In [4]:
from datasets import load_dataset
questions = load_dataset("rag-datasets/rag-mini-wikipedia", "question-answer")
text = load_dataset("rag-datasets/rag-mini-wikipedia", "text-corpus")

In [5]:
print(questions)
print(text)

DatasetDict({
    test: Dataset({
        features: ['question', 'answer', 'id'],
        num_rows: 918
    })
})
DatasetDict({
    passages: Dataset({
        features: ['passage', 'id'],
        num_rows: 3200
    })
})


In [6]:
from datasets import DatasetDict

questions_all = questions["test"]

# split = questions_all.train_test_split(test_size=0.2, seed=42)
# questions_train = split["train"]
# temp = split["test"].train_test_split(test_size=0.5, seed=42)
# questions_val = temp["train"]
# questions_test = temp["test"]

# questions_split = DatasetDict({
#     "train": questions_train,
#     "validation": questions_val,
#     "test": questions_test
# })

questions_val = questions_all
questions_split = DatasetDict({
    "validation": questions_val
})

In [7]:
print(questions_split)

DatasetDict({
    validation: Dataset({
        features: ['question', 'answer', 'id'],
        num_rows: 918
    })
})


## Constantes

In [8]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16

## Encodeur

### Chargement de l'encodeur

In [9]:
from transformers import AutoTokenizer, AutoModel
model_name = 'BAAI/bge-base-en-v1.5'
emb_tokenizer = AutoTokenizer.from_pretrained(model_name)
emb_model = AutoModel.from_pretrained(model_name)
emb_model.to(DEVICE)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

### Stockage dans FAISS

In [10]:
def encode_sequences(sequences: list, tokenizer, model, device=DEVICE, batch_size=BATCH_SIZE, training=False) -> torch.Tensor:
    model.train() if training else model.eval()
    context = torch.enable_grad() if training else torch.no_grad()

    embeddings = []
    with context:
        for i in range(0, len(sequences), batch_size):
            batch = sequences[i:i + batch_size]

            # Tokenisation des textes avant entrée du modèle
            inputs = tokenizer(
                batch, padding=True, truncation=True, return_tensors="pt"
            ).to(device)

            # Passage dans le modèle encodeur
            output = model(**inputs).last_hidden_state

            # Pooling : moyenne des vecteurs de tous les tokens pour chaque séquence
            pooled = output.mean(dim=1)  # [batch_size, hidden_size]

            embeddings.append(pooled)

    # Concatène tous les embeddings pour obtenir un seul tensor de sortie
    return torch.cat(embeddings, dim=0)

In [11]:
def retrieve_top_k_passages_faiss(questions: list, vector_index: faiss.IndexFlatL2, embedding_model_tokenizer, embedding_model, k: int):
    question_embeddings = encode_sequences(questions, embedding_model_tokenizer, embedding_model)
    question_embeddings = question_embeddings.cpu().numpy()
    faiss.normalize_L2(question_embeddings)
    _, top_k_indices = vector_index.search(question_embeddings, k)
    
    return top_k_indices

In [12]:
passages = text["passages"]["passage"]
passage_embed = encode_sequences(passages, emb_tokenizer, emb_model)
passage_embeddings = passage_embed.cpu().numpy()
faiss.normalize_L2(passage_embeddings)

d = passage_embeddings.shape[1]
index = faiss.IndexFlatIP(d)
index.add(passage_embeddings)

## HyDE

## Generator

### Chargement du générateur

In [ ]:
from huggingface_hub import login
login(token = '')

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

gen_tokenizer = AutoTokenizer.from_pretrained(model_name)
gen_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",           # auto-placement sur GPU si dispo
    torch_dtype=torch.float16,   # allège la mémoire (fp16)
    trust_remote_code=True,       # pour activer les modèles qui utilisent du code perso
)
gen_tokenizer.pad_token = gen_tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### HyDE

In [16]:
def generate_prompts_no_context(questions):
    prompts = []
    for question in questions:
        prompt = (
            "Answer precisely to the following question.\n"
            f"Question: {question}\n\nAnswer:"
        )
        prompts.append(prompt)
    return prompts

In [17]:
questions_val = questions_split["validation"]["question"]
prompts_no_context = generate_prompts_no_context(questions_val)

print(prompts_no_context)

['Answer precisely to the following question.\nQuestion: Was Abraham Lincoln the sixteenth President of the United States?\n\nAnswer:', 'Answer precisely to the following question.\nQuestion: Did Lincoln sign the National Banking Act of 1863?\n\nAnswer:', 'Answer precisely to the following question.\nQuestion: Did his mother die of pneumonia?\n\nAnswer:', "Answer precisely to the following question.\nQuestion: How many long was Lincoln's formal education?\n\nAnswer:", 'Answer precisely to the following question.\nQuestion: When did Lincoln begin his political career?\n\nAnswer:', 'Answer precisely to the following question.\nQuestion: What did The Legal Tender Act of 1862 establish?\n\nAnswer:', 'Answer precisely to the following question.\nQuestion: Who suggested Lincoln grow a beard?\n\nAnswer:', 'Answer precisely to the following question.\nQuestion: When did the Gettysburg address argue that America was born?\n\nAnswer:', 'Answer precisely to the following question.\nQuestion: Did 

In [18]:
def process_batch_HyDE_prompts(batch_prompts, tokenizer, model, device, max_new_tokens=64):
    formatted_prompts = [
        f"<s>[INST] {prompt.strip()} [/INST]" for prompt in batch_prompts
    ]

    print(formatted_prompts)

    inputs = tokenizer(formatted_prompts, return_tensors="pt", padding=True, truncation=True).to(device)

    # Génération
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id
    )

    # Décodage et extraction de la réponse
    responses = [
        tokenizer.decode(output, skip_special_tokens=True).split('[/INST]')[-1].strip()
        for output in outputs
    ]

    print(responses)

    return responses

def chunked_iterable(iterable, size):
    it = iter(iterable)
    while True:
        chunk = tuple(itertools.islice(it, size))
        if not chunk:
            return
        yield chunk

def generate_hypothetical_docs(gen_model, gen_tokenizer, prompts, batch_size = 32) : 
    batch_responses = []
    
    for batch_index, batch_prompts in enumerate(chunked_iterable(prompts, batch_size)):
        print(f"Batch {batch_index}] Started generation...")
        start_time = time.time()
        
        batch_responses.extend(process_batch_HyDE_prompts(batch_prompts, gen_tokenizer, gen_model, DEVICE))
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"[Batch {batch_index}] Time for generation = {elapsed_time:.2f} seconds")
    return batch_responses

In [19]:
hyde_docs = generate_hypothetical_docs(gen_model, gen_tokenizer, prompts_no_context)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Batch 0] Started generation...
['<s>[INST] Answer precisely to the following question.\nQuestion: Was Abraham Lincoln the sixteenth President of the United States?\n\nAnswer: [/INST]', '<s>[INST] Answer precisely to the following question.\nQuestion: Did Lincoln sign the National Banking Act of 1863?\n\nAnswer: [/INST]', '<s>[INST] Answer precisely to the following question.\nQuestion: Did his mother die of pneumonia?\n\nAnswer: [/INST]', "<s>[INST] Answer precisely to the following question.\nQuestion: How many long was Lincoln's formal education?\n\nAnswer: [/INST]", '<s>[INST] Answer precisely to the following question.\nQuestion: When did Lincoln begin his political career?\n\nAnswer: [/INST]', '<s>[INST] Answer precisely to the following question.\nQuestion: What did The Legal Tender Act of 1862 establish?\n\nAnswer: [/INST]', '<s>[INST] Answer precisely to the following question.\nQuestion: Who suggested Lincoln grow a beard?\n\nAnswer: [/INST]', '<s>[INST] Answer precisely to th

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['Yes, Abraham Lincoln was the sixteenth President of the United States. He served from 1861 until his assassination in 1865.', 'No, Lincoln did not sign the National Banking Act of 1863. The National Banking Act was signed into law by President Andrew Jackson in 1832. The misconception may arise due to the similarity in the names of the two acts and the fact that Lincoln was a well', "I don't have access to personal or medical information about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. Therefore, I cannot provide an answer to this question.", 'Abraham Lincoln, the 16th President of the United States, received a total of 12 years of formal education. He attended a one-room school in Hardin County, Kentucky for one year when he was six years old. After that, he received the remainder of his formal education in a self-', 'Abraham Lincoln began his political career in 1832 when he was elected

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

['No, sea turtles do not lay their eggs on dry sandy beaches. They lay their eggs in nests on sandy beaches, but the eggs are not dry. The eggs are covered with a protective coating that keeps them moist.', "Yes, Pleurodira are commonly known as side-necked turtles. This is because of the unique shape of their necks, which are flatter on one side and more curved on the other, resembling the side of a ship's neck.", 'No, turtles are not a part of the Flying Spaghetti Monster. The Flying Spaghetti Monster is a mythical creature often depicted as a giant, flying, spaghetti-like creature with meatballs for eyes. It is not a real animal, and', 'Turtles are divided into two suborders: Testudines and Cryptodira.', 'One turtle that is popularly kept as a pet is the common box turtle (Rubber Ducky).', '1. "The Biology and Behavior of Turtles"\n2. "Turtle Conservation and Protection"\n3. "Turtles as Pets: Care and Handling"\n4. "Turtle Diet and Nutrition"', 'If a turtle egg was kept warm, it wou

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


["Ulysses S. Grant's brother-in-law was Simon Bolivar Buckley.", 'Grant was born in Point Pleasant, Ohio.', 'Ulysses S. Grant was a Republican.', 'Grant said "Damn, I had nothing to do with this batte." because he was falsely accused of being involved in a battle and wanted to clarify that he had no participation in it.', 'Yes, Horace Greeley lost the presidential election of 1872. He was the candidate of the Liberal Republican Party and ran against Ulysses S. Grant, who was seeking re-election as the Republican Party nominee. Grant won the election, receiving 212 electoral votes', 'Ulysses S. Grant, the 18th President of the United States, did not express any public statements indicating that he thought war was unjust during his time in office. However, it is worth noting that after the Civil War, Grant wrote his memoirs, titled "The Personal Memoirs of', 'General Ulysses S. Grant achieved international fame as the leading Union general in the American Civil War.', 'Grant was elected 

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['Uruguay is a country in South America, and it has a total land area of approximately 28,284 square kilometers. Of this total land area, 97.8% is continental land, while the remaining 2.2% is coastal and island land. Therefore, the', 'The percentage of the global population that is of white European descent varies widely depending on the source and the specific definition of "white European descent." However, according to the United Nations, the global population with European ancestry was estimated to be around 1.1 billion in 2019, which accounts for approximately', "The capital of Uruguay, Montevideo, was founded on December 15, 1819, as a result of the country's independence from Spain. It was strategically located on the eastern coast of the country, at the mouth of the Sacramento River, and served as a key", 'Uruguay has won the FIFA World Cup twice. They first won the tournament in 1930, hosting the event in Montevideo. They then won again in 2010, in South Africa.', 'I cannot p

### Answer Prompts

In [ ]:
def generate_prompts_answer(docs, questions, index, emb_tokenizer, emb_model, k, passages):
    prompts = []
    faiss_passages = retrieve_top_k_passages_faiss(docs, index, emb_tokenizer, emb_model, k)
    help_instruction = "Give an answer to the question using one word if possible."

    for i, passages_ids in enumerate(faiss_passages):
        # Récupérer les textes directement via les indices
        retrieved_passages = [passages[j] for j in passages_ids]

        # Construire le prompt
        context_string = (
            f"{help_instruction}\n" +
            "\n".join(retrieved_passages) +
            "\n" +
            f"Question: {questions[i]}\n"
        )
        prompts.append(context_string)

    return prompts

In [21]:
questions_val = questions_split["validation"]["question"]
passages = text["passages"]["passage"]

k_values = [1, 2, 3, 4]
prompts_k = {}

for k in k_values:
    prompts_k[k] = generate_prompts_answer(hyde_docs, questions_val, index, emb_tokenizer, emb_model, k, passages)

# Affichage exemple
print(prompts_k[3][0])

Give an answer to the question using one word if possible.
Abraham Lincoln (February 12, 1809 â April 15, 1865) was the sixteenth President of the United States, serving from March 4, 1861 until his assassination. As an outspoken opponent of the expansion of slavery in the United States, "[I]n his short autobiography written for the 1860 presidential campaign, Lincoln would describe his protest in the Illinois legislature as one that 'briefly defined his position on the slavery question, and so far as it goes, it was then the same that it is now." This was in reference to the anti-expansion sentiments he had then expressed. Doris Kearns Goodwin, Team of Rivals: The Political Genius of Abraham Lincoln (2005) p. 91.  Holzer pg. 232.  Writing of the Cooper Union  speech, Holzer notes, "Cooper Union proved a unique confluence of political culture, rhetorical opportunity, technological innovation, and human genius, and it brought Abraham Lincoln to the center stage of American politics at

### Génération

In [22]:
def process_batch_prompts_answer(batch_prompts, tokenizer, model, device, max_new_tokens=64):
    formatted_prompts = [
        f"<s>[INST] {prompt.strip()} [/INST]" for prompt in batch_prompts
    ]

    print(formatted_prompts)

    inputs = tokenizer(formatted_prompts, return_tensors="pt", padding=True, truncation=True).to(device)

    # Génération
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        do_sample=False,
        # top_p=0.9,
        # temperature=0.7,
        eos_token_id=tokenizer.eos_token_id
    )

    # Décodage et extraction de la réponse
    responses = [
        tokenizer.decode(output, skip_special_tokens=True).split('[/INST]')[-1].strip()
        for output in outputs
    ]

    print(responses)

    return responses

def chunked_iterable(iterable, size):
    it = iter(iterable)
    while True:
        chunk = tuple(itertools.islice(it, size))
        if not chunk:
            return
        yield chunk

def generate_responses(gen_model, gen_tokenizer, prompts, batch_size = 32) : 
    batch_responses = []
    
    for batch_index, batch_prompts in enumerate(chunked_iterable(prompts, batch_size)):
        print(f"[k={k} | Batch {batch_index}] Started generation...")
        start_time = time.time()
        
        batch_responses.extend(process_batch_prompts_answer(batch_prompts, gen_tokenizer, gen_model, DEVICE))
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"[k={k} | Batch {batch_index}] Time for generation = {elapsed_time:.2f} seconds")
    return batch_responses

In [23]:
k_values = [3]
generated_responses = {}
for k in k_values:
    generated_responses[k] = generate_responses(gen_model, gen_tokenizer, prompts_k[k])
    print(f"Done with k = {k}!")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[k=3 | Batch 0] Started generation...
['<s>[INST] Give an answer to the question using one word if possible.\nAbraham Lincoln (February 12, 1809 â\x80\x93 April 15, 1865) was the sixteenth President of the United States, serving from March 4, 1861 until his assassination. As an outspoken opponent of the expansion of slavery in the United States, "[I]n his short autobiography written for the 1860 presidential campaign, Lincoln would describe his protest in the Illinois legislature as one that \'briefly defined his position on the slavery question, and so far as it goes, it was then the same that it is now." This was in reference to the anti-expansion sentiments he had then expressed. Doris Kearns Goodwin, Team of Rivals: The Political Genius of Abraham Lincoln (2005) p. 91.  Holzer pg. 232.  Writing of the Cooper Union  speech, Holzer notes, "Cooper Union proved a unique confluence of political culture, rhetorical opportunity, technological innovation, and human genius, and it brought A

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['Yes.', 'Yes.', 'No.', '18 months.', '1832.', 'Paper currency.', 'Grace Bedell', 'November 19, 1863.', 'Yes.', 'No. Abraham Lincoln was the sixteenth President of the United States.', 'Yes.', 'Yes.', 'Hardin', 'March 4, 1861.', 'Booth.', 'Yes.', 'McClellan', 'To free slaves in Confederate-held territories.', 'Yes.', 'Yes.', 'Yes.', 'Seven.', '1846.', "Lincoln's legal career spanned from 1837 to 1861.", 'Court trial.', 'Yes.', 'No.', 'Avogadro.', 'John Adams', 'No.', 'Not specified.', 'No.']
[k=3 | Batch 0] Time for generation = 4.71 seconds
[k=3 | Batch 1] Started generation...
['<s>[INST] Give an answer to the question using one word if possible.\nLorenzo Romano Amedeo Carlo Avogadro, Count of Quaregna and Cerreto (August 9, 1776 July 9, 1856) was an Italian savant.  He is most noted for his contributions to the theory of molarity and molecular weight. As a tribute to him, the number of elementary entities (atoms, molecules, ions or other particles) in one mole of a substance, 6.0221

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

['Yes.', 'Yes.', 'No.', 'Three.', 'Turtle.', '1. Encyclopedia of Turtles by Pritchard, Pether C H (1979)\n2. Turtles and Crocodiles of Insular Southeast Asia and New Guinea by Iskandar, DT (2000)\n3. An', 'Female.', 'Papillae.', 'Reference', 'No.', 'Yes.', 'Yes.', 'Paracryptodira.', 'Yolk.', 'Beaches.', 'Pleurodira.', 'Sand.', 'Reptiles.', 'Water.', 'All turtles are ectothermic.', 'Yes.', 'Yes.', 'Papillae are not mentioned in the provided text.', 'Ostrich.', 'Yes.', 'Yes.', 'Yes.', 'Yes.', 'Yes.', 'Yes.', 'Yes.', 'Yes.']
[k=3 | Batch 25] Time for generation = 6.15 seconds
[k=3 | Batch 26] Started generation...
["<s>[INST] Give an answer to the question using one word if possible.\nUlysses S. Grant in his postbellum.\nLt. Gen. Ulysses S. Grant\nPresident Grant with his wife, Julia, and son, Jesse, in 1872.\nQuestion: Who was Grant's brother in law? [/INST]", '<s>[INST] Give an answer to the question using one word if possible.\nUlysses Grant Birthplace, Point Pleasant, Ohio\nGrant was 

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['None.', 'Point Pleasant.', 'Republican', 'Unclear.', 'Yes.', 'No answer provided as there is no information given in the question that suggests Ulysses S. Grant thought war was unjust.', 'Ulysses S. Grant', '1868', 'Grant', 'No information provided.', 'Grant', 'Yes.', 'Pennsylvania', 'No.', 'Yes.', 'No.', 'No.', 'Yes.', 'The American Civil War ended in 1865.', 'Not specified.', 'No.', 'Yes.', 'No.', 'Spanish.', 'San Carlos.', 'Europeans.', 'Cantegriles.', 'No.', 'Catholic.', 'Yes.', 'Yes.', 'Yes.']
[k=3 | Batch 26] Time for generation = 4.13 seconds
[k=3 | Batch 27] Started generation...
['<s>[INST] Give an answer to the question using one word if possible.\nAt 176,214 square kilometres (68,036 square miles) of continental land and 142,199 square kilometres (54,903 sq mi) of jurisdictional waters and small river islands,  Instituto Nacional Estadistica  Uruguay is the second smallest sovereign nation in South America (after Suriname) and the third smallest territory (French Guiana is

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['176,214', '88%', 'Founded.', 'Two.', '65%.', 'Yes.', 'No.', 'Yes.', 'Plebiscite.', 'Catholicism, Protestantism, Judaism.', 'Uruguay has been competitive in agriculture by focusing on low inputs of labor, technology, and capital, resulting in comparatively lower yields per hectare but opening the door for marketing products as "natural" or "ecological."', 'Partido Colorado, Partido Blanco, Frente Amplio.', 'No.', 'Montevideo', 'Language.', 'Diverse.', 'Independence.', 'No.', 'Yes.', 'Yes.', 'Yes.', 'Yes.', 'Yes.', 'Yes.', 'No.', 'No.', 'Yes.', 'No.', 'Paris', 'December 28, 1856.', 'Wilson was President when he finished writing Congressional Government.', 'History and political science.']
[k=3 | Batch 27] Time for generation = 6.60 seconds
[k=3 | Batch 28] Started generation...
['<s>[INST] Give an answer to the question using one word if possible.\nWoodrow Wilson had decided by then that the war had become a real threat to humanity. Unless the U.S. threw its weight into the war, as he 

## Evaluation

In [26]:
questions_val = questions_split["validation"].to_pandas()
df_export = questions_val[['id', 'question', 'answer']].copy()
df_export['generated_answer'] = generated_responses[k]
df_export.to_csv(f"generated_responses_simple_HyDE_v2.csv", index=False, encoding='utf-8')